In [2]:
# hide warnings to keep things tidy.
import warnings
warnings.filterwarnings('ignore')
import random

import numpy as np

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline  
matplotlib.style.use('ggplot') # make things a bit prettier.

import pandas as pd
import statsmodels.api as sm 
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsRegressor
import random
from numpy.random import permutation
import math
from sklearn import metrics
names = ['UserID', 'MovieID', 'Rating', 'Timestamp']
df = pd.read_csv("/Users/sushma/Desktop/DM_Git/Recommender_System/newRatings.csv", names=names, header=0)
df.head()

,UserID,MovieID,Rating,Timestamp
0,1,661,3,978302109
1,1,914,3,978301968
2,1,3408,4,978300275
3,1,1287,5,978302039
4,1,594,4,978302268


In [3]:
n_users = df.UserID.unique().shape[0]
print("n users: %s" % n_users)
n_movies = df.MovieID.unique().shape[0]
print("n movies: %s" % n_movies)
max_movies = df.MovieID.unique().max()
print max_movies

n users: 6040
n movies: 1970
3951


In [4]:
ratings = np.zeros((n_users,max_movies))
for row in df.itertuples():
    # row[1] will the user_id; row[2] the 
    # rating (see also my note below).
    # the -1 is to move to 0 indexing.
    ratings[row[1]-1, row[2]-1] = row[3]
ratings

array([[ 5.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 3.,  0.,  0., ...,  0.,  0.,  0.]])

In [5]:
print ratings.shape[0]

6040


In [6]:
dfm1 = pd.read_csv("/Users/sushma/Desktop/DM_Git/Recommender_System/FirstCluster.csv", header=0)

dfm1.head()

,UserID,MovieID,Rating,label
0,1,661,3,0
1,1,914,3,0
2,1,594,4,0
3,1,938,4,0
4,1,2687,3,0


The code below is used to only select the portion of ratings file that belongs to first cluster. I have already generated the csv and uploaded it to our git. So you need not repeat this step. It is very time consuming

In [ ]:
dc = pd.read_csv('/Users/sushma/Desktop/DM_Git/Recommender_System/Movies_cluster1.csv', header=0)
dc.head()

In [ ]:
first_cluster = []
for row in df.itertuples():
    for l in dc.itertuples():
        if row[2] == l[2]:
            first_cluster.append([row[1],row[2],row[3],l[9]])

In [ ]:
import csv

with open('/Users/sushma/Desktop/DM_Git/Recommender_System/FirstCluster.csv', 'a') as outcsv:   
    writer = csv.writer(outcsv, delimiter=',', quoting=csv.QUOTE_MINIMAL, lineterminator='\n')
    writer.writerow(['UserID','MovieID','Rating','label'])
    for item in first_cluster:
        writer.writerow([item[0], item[1], item[2], item[3]])  

Once we have the csv file for first cluster, we are going to find all the userID's of first cluster and extract only rows that contain a matching userID from the ratings similarity matrix. Basically, we are first clutering our movies dataset and now creating a similarity matrix for only first cluster so that we can apply knn

In [7]:
dfm1 = pd.read_csv("/Users/sushma/Desktop/DM_Git/Recommender_System/FirstCluster.csv", header=0)
dfm1.head()

,UserID,MovieID,Rating,label
0,1,661,3,0
1,1,914,3,0
2,1,594,4,0
3,1,938,4,0
4,1,2687,3,0


In [8]:
unique_userid = dfm1.UserID.unique()
unique_userid_length = dfm1.UserID.unique().shape[0]
print unique_userid
print unique_userid_length

[   1    2    3 ..., 6038 6039 6040]
6037


matrix_cluster1 is the similarity matrix extracted from ratings similarity matrix and represents the data that belongs to first cluster only.

In [9]:
list_cluster1 = []
for rowid in range(ratings.shape[0]):
    if rowid in unique_userid:
        list_cluster1.append(ratings[rowid, :])  
matrix_cluster1 = np.array(list_cluster1)
matrix_cluster1

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 3.,  0.,  0., ...,  0.,  0.,  0.]])

In [148]:
matrix_cluster1[5,6]
matrix_cluster1.shape

(6036, 3951)

In [149]:
clust1_train, clust1_test = train_test_split(matrix_cluster1)

In [150]:
clust1_user_similarity = similarity(clust1_train, kind='user')
clust1_item_similarity = similarity(clust1_train, kind='item')

[[  1.03800000e+03   1.27000000e+02   7.70000000e+01 ...,   2.00000000e+01
    8.50000000e+01   3.71000000e+02]
 [  1.27000000e+02   3.83000000e+02   3.70000000e+01 ...,   1.00000000e-09
    6.60000000e+01   1.81000000e+02]
 [  7.70000000e+01   3.70000000e+01   1.09000000e+02 ...,   1.00000000e-09
    1.50000000e+01   6.10000000e+01]
 ..., 
 [  2.00000000e+01   1.00000000e-09   1.00000000e-09 ...,   6.60000000e+01
    3.70000000e+01   1.90000000e+01]
 [  8.50000000e+01   6.60000000e+01   1.50000000e+01 ...,   3.70000000e+01
    9.92000000e+02   2.69000000e+02]
 [  3.71000000e+02   1.81000000e+02   6.10000000e+01 ...,   1.90000000e+01
    2.69000000e+02   2.58200000e+03]]


Mean squared errror for predictions in first cluster

In [151]:
clust1_item_prediction = predict(clust1_train, clust1_item_similarity, kind='item')
clust1_user_prediction = predict(clust1_train, clust1_user_similarity, kind='user')

print('User-based CF MSE: ' + str(get_mse(clust1_user_prediction, clust1_test)))
print('Item-based CF MSE: ' + str(get_mse(clust1_item_prediction, clust1_test)))

User-based CF MSE: 9.40684991455
Item-based CF MSE: 12.8792060007


The code below is similar to what we learned in class. Creating a similarity matrix for the entire ratings csv (newRatings) and predicting using knn

In [82]:
sparsity = float(len(ratings.nonzero()[0]))
sparsity /= (ratings.shape[0] * ratings.shape[1])
sparsity *= 100
print('sparsity: {:4.2f}%'.format(sparsity))

sparsity: 2.43%


In [86]:
def train_test_split(ratings):
    test = np.zeros(ratings.shape)
    train = ratings.copy()
    for user in range(ratings.shape[0]):
        # sample 10 ratings from each user to use
        # as 'test' data
        test_ratings = np.random.choice(ratings[user, :].nonzero()[0], 
                                            size=6, 
                                            replace=False)
        # effectively remove these from the training
        # set
        train[user, test_ratings] = 0.
        test[user, test_ratings] = ratings[user, test_ratings]
           
    # make sure that test and training are truly disjoint
    assert(np.all((train * test) == 0)) 
    
    return train, test

In [142]:
train, test = train_test_split(ratings)

In [88]:
def similarity(ratings, kind='user', epsilon=1e-9):
    # epsilon -> small number for handling dived-by-zero errors
    if kind == 'user':
        sim = ratings.dot(ratings.T) + epsilon
        print(sim)
    elif kind == 'item':
        # we need only flip the dimensions around 
        # to do item based similarity!
        sim = ratings.T.dot(ratings) + epsilon
    norms = np.array([np.sqrt(np.diagonal(sim))])
    return (sim / (norms * norms.T))

In [89]:
user_similarity = similarity(train, kind='user')
item_similarity = similarity(train, kind='item')

[[  4.10000000e+02   4.50000000e+01   3.50000000e+01 ...,   1.00000000e-09
    6.80000000e+01   1.44000000e+02]
 [  4.50000000e+01   1.06800000e+03   1.37000000e+02 ...,   1.50000000e+01
    5.60000000e+01   3.58000000e+02]
 [  3.50000000e+01   1.37000000e+02   3.95000000e+02 ...,   1.00000000e-09
    4.00000000e+01   1.35000000e+02]
 ..., 
 [  1.00000000e-09   1.50000000e+01   1.00000000e-09 ...,   5.90000000e+01
    2.50000000e+01   2.50000000e+01]
 [  6.80000000e+01   5.60000000e+01   4.00000000e+01 ...,   2.50000000e+01
    9.76000000e+02   3.03000000e+02]
 [  1.44000000e+02   3.58000000e+02   1.35000000e+02 ...,   2.50000000e+01
    3.03000000e+02   2.59200000e+03]]


In [90]:
sim = ratings.dot(train.T) + 1e-9
sim

array([[  4.10000000e+02,   6.50000000e+01,   3.50000000e+01, ...,
          1.00000000e-09,   9.70000000e+01,   1.56000000e+02],
       [  6.50000000e+01,   1.06800000e+03,   1.37000000e+02, ...,
          1.50000000e+01,   1.01000000e+02,   4.02000000e+02],
       [  3.50000000e+01,   1.49000000e+02,   3.95000000e+02, ...,
          1.00000000e-09,   4.60000000e+01,   1.69000000e+02],
       ..., 
       [  1.00000000e-09,   5.10000000e+01,   1.60000000e+01, ...,
          5.90000000e+01,   3.70000000e+01,   8.00000000e+01],
       [  8.00000000e+01,   7.60000000e+01,   6.00000000e+01, ...,
          2.50000000e+01,   9.76000000e+02,   3.13000000e+02],
       [  1.44000000e+02,   3.73000000e+02,   1.47000000e+02, ...,
          2.50000000e+01,   3.03000000e+02,   2.59200000e+03]])

In [91]:
def predict(ratings, similarity, kind='user'):
    if kind == 'user':
        return similarity.dot(ratings) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif kind == 'item':
        return ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])

In [92]:
from sklearn.metrics import mean_squared_error
def get_mse(pred, actual):
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

In [93]:
item_prediction = predict(train, item_similarity, kind='item')
user_prediction = predict(train, user_similarity, kind='user')

print('User-based CF MSE: ' + str(get_mse(user_prediction, test)))
print('Item-based CF MSE: ' + str(get_mse(item_prediction, test)))

User-based CF MSE: 9.41283858874
Item-based CF MSE: 12.8813660348


In [16]:
def predict_topk(ratings, similarity, kind='user', k=40):
    pred = np.zeros(ratings.shape)
    
    for i in range(ratings.shape[0]):
        # note! the [::-1] craziness is just a numpy trick to 
        # reverse the array. we're doing this because otherwise
        # it is sorted in the direction opposite of what we want
        # -- lowest to highest, so least to most similar. 
        # we flip this around then take the top k.
        top_k_users =  np.argsort(similarity[:,i])[::-1][:k] #[np.argsort(similarity[:,i])[:-k-1:-1]]
        
        for j in range(ratings.shape[1]):
            pred[i, j] = similarity[i, :][top_k_users].dot(ratings[:, j][top_k_users]) 
            pred[i, j] /= np.sum(np.abs(similarity[i, :][top_k_users]))

    return pred

In [17]:
pred = predict_topk(train, user_similarity, k=40)
print('Top-k User-based CF MSE: ' + str(get_mse(pred, test)))

Top-k User-based CF MSE: 7.94329177034


In [ ]:
k_array = [5, 15, 30, 50, 100, 200]
user_train_mse = []
user_test_mse = []
item_test_mse = []
item_train_mse = []

def get_mse(pred, actual):
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

for k in k_array:
    user_pred = predict_topk(train, user_similarity, k=k)
    
    user_train_mse += [get_mse(user_pred, train)]
    user_test_mse += [get_mse(user_pred, test)]

In [ ]:
pal = sns.color_palette("Set2", 2)

plt.figure(figsize=(8, 8))
plt.plot(k_array, user_train_mse, c=pal[0], label='User-based train', alpha=0.5, linewidth=5)
plt.plot(k_array, user_test_mse, c=pal[0], label='User-based test', linewidth=5)

plt.legend(loc='best', fontsize=20)
plt.xticks(fontsize=16);
plt.yticks(fontsize=16);
plt.xlabel('k', fontsize=30);
plt.ylabel('MSE', fontsize=30);